<h1>Fundaments of Data Visualization - Final Project</h1>

Description: This code creates a visualization of open payment data using altair.

Author: Alan Klein

Create Date: 2023/10/15

In [2]:
# Imports used modules, may need to install.
import pandas as pd
import altair as alt

In [3]:
#  Define the columns I'm interested in from the data

General_Payment_Data_Columns = ['Covered_Recipient_Type',
#'Covered_Recipient_Profile_ID',
'Covered_Recipient_NPI',
'Covered_Recipient_First_Name',
'Covered_Recipient_Middle_Name',
'Covered_Recipient_Last_Name',
'Covered_Recipient_Name_Suffix',
#'Recipient_Primary_Business_Street_Address_Line1',
#'Recipient_Primary_Business_Street_Address_Line2',
'Recipient_City',
'Recipient_State',
#'Recipient_Zip_Code',
#'Recipient_Country',
#'Recipient_Province',
#'Recipient_Postal_Code',
#'Covered_Recipient_Primary_Type_1',
'Covered_Recipient_Specialty_1',
#'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
#'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
'Total_Amount_of_Payment_USDollars',
#'Date_of_Payment',
'Number_of_Payments_Included_in_Total_Amount',
'Form_of_Payment_or_Transfer_of_Value',
'Nature_of_Payment_or_Transfer_of_Value',
#'Related_Product_Indicator',
#'Covered_or_Noncovered_Indicator_1',
'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
'Product_Category_or_Therapeutic_Area_1',
'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
#'Associated_Drug_or_Biological_NDC_1',
#'Associated_Device_or_Medical_Supply_PDI_1'
]

In [4]:
#load open payment data and reference data.  
#Open payment data is payments made from pharma companies to physicains.  
#More information can be found here: https://www.cms.gov/priorities/key-initiatives/open-payments

#Dataset is huge so needs to be loaded iteratively and filtered

iter_csv = pd.read_csv("./Data/OP_DTL_GNRL_PGYR2022_P06302023.csv", iterator=True, chunksize=10000, usecols = General_Payment_Data_Columns)
General_Payment_Data_MN_Drug = pd.concat([chunk[(chunk['Recipient_State'] == 'MN') & (chunk['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug') & (chunk['Covered_Recipient_Type'] == 'Covered Recipient Physician')] for chunk in iter_csv])






C:\Users\ajkle\AppData\Local\Temp\ipykernel_2476\4246699200.py:8: DtypeWarning: Columns (60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  General_Payment_Data_MN_Drug = pd.concat([chunk[(chunk['Recipient_State'] == 'MN') & (chunk['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug') & (chunk['Covered_Recipient_Type'] == 'Covered Recipient Physician')] for chunk in iter_csv])
C:\Users\ajkle\AppData\Local\Temp\ipykernel_2476\4246699200.py:8: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  General_Payment_Data_MN_Drug = pd.concat([chunk[(chunk['Recipient_State'] == 'MN') & (chunk['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug') & (chunk['Covered_Recipient_Type'] == 'Covered Recipient Physician')] for chunk in iter_csv])
C:\Users\ajkle\AppData\Local\Temp\ipykernel_2476\4246699200.py:8: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on impo

In [5]:

#Create Full Name from First Middle and Last
General_Payment_Data_MN_Drug['Physician_Name'] = General_Payment_Data_MN_Drug[['Covered_Recipient_Last_Name', 'Covered_Recipient_First_Name']].apply(lambda x : x.str.cat(sep=', '),axis=1)

#Extract the final part of physician specialty
General_Payment_Data_MN_Drug['Physician_Specialty_Short'] = General_Payment_Data_MN_Drug['Covered_Recipient_Specialty_1'].str.extract(r'([^|]*)$')

#Converting everything to uppercase
General_Payment_Data_MN_Drug['Physician_Name'] = General_Payment_Data_MN_Drug['Physician_Name'].str.upper()
General_Payment_Data_MN_Drug['Physician_Specialty_Short'] = General_Payment_Data_MN_Drug['Physician_Specialty_Short'].str.upper()
General_Payment_Data_MN_Drug['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'] = General_Payment_Data_MN_Drug['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].str.upper()
General_Payment_Data_MN_Drug['Form_of_Payment_or_Transfer_of_Value'] = General_Payment_Data_MN_Drug['Form_of_Payment_or_Transfer_of_Value'].str.upper()
General_Payment_Data_MN_Drug['Nature_of_Payment_or_Transfer_of_Value'] = General_Payment_Data_MN_Drug['Nature_of_Payment_or_Transfer_of_Value'].str.upper()
General_Payment_Data_MN_Drug['Product_Category_or_Therapeutic_Area_1'] = General_Payment_Data_MN_Drug['Product_Category_or_Therapeutic_Area_1'].str.upper()
General_Payment_Data_MN_Drug['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'] = General_Payment_Data_MN_Drug['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'].str.upper()

#List of columns to keep to feed into altair
General_Payment_Data_Columns_Filtered = ['Physician_Name',
'Covered_Recipient_NPI',
'Physician_Specialty_Short',
'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
'Total_Amount_of_Payment_USDollars',
'Number_of_Payments_Included_in_Total_Amount',
#'Form_of_Payment_or_Transfer_of_Value',
'Nature_of_Payment_or_Transfer_of_Value',
'Product_Category_or_Therapeutic_Area_1',
'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'
]

#Limit dataframe to list of columns
General_Payment_Data_MN_Drug_Full_Name = General_Payment_Data_MN_Drug[General_Payment_Data_Columns_Filtered]

#Rollup data
Group_By_Columns = ['Physician_Name',
'Covered_Recipient_NPI',
'Physician_Specialty_Short',
'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
#'Form_of_Payment_or_Transfer_of_Value',
'Nature_of_Payment_or_Transfer_of_Value',
'Product_Category_or_Therapeutic_Area_1',
'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'
]

GPD_Aggregated = General_Payment_Data_MN_Drug_Full_Name.groupby(Group_By_Columns).agg({'Total_Amount_of_Payment_USDollars': ["sum"], 'Number_of_Payments_Included_in_Total_Amount': ["sum"]})

#Flatten Data
GPD_Aggregated = GPD_Aggregated.droplevel(axis=1, level=0).reset_index()

#Rename Columns
GPD_Aggregated.columns =   ['Physician_Name', 
                            'NPI', 
                            'Physician_Specialty', 
                            'Company', 
                            #'Form_of_Payment',
                            'Nature_of_Payment', 
                            'Therapeutic_Area', 
                            'Drug', 
                            'Total_Payment',
                            'Number_of_Payments']



In [6]:
#Write a CSV to reduce data size
GPD_Aggregated.to_csv('./Open_Payment_Data_For_Final.csv')

In [36]:
#Add a dropdown to search for and select provider
dropdown = alt.binding_select(options=GPD_Aggregated['Physician_Name'].sort_values().unique(), name="Select a Physician:")

#Add ability to select providers
selection = alt.selection_point(fields=['Physician_Name'], bind = dropdown)

# Create a base charte size and data
base =  alt.Chart(GPD_Aggregated).properties(width=250, height=250)

#Main scatter plot
scatterplot = base.mark_circle().encode(
    x ='Number_of_Payments',
    y ='Total_Payment',
    color=alt.Color('Nature_of_Payment', scale=alt.Scale(scheme='spectral')),
    tooltip=['Physician_Name', 'NPI', 'Physician_Specialty'],
    opacity=alt.condition(selection,alt.value(1),alt.value(.1))
).add_params(selection).properties(width=500, height=500)

# Decided to not use this bar chat as I didn't think it added anything
#barchart = base.mark_bar().encode(
#    x = 'Nature_of_Payment',
#    y = 'Total_Payment',
#    color=alt.Color('Nature_of_Payment', scale=alt.Scale(scheme='spectral')),
#    tooltip= ['Number_of_Payments']
#).transform_filter(selection)

# Data on what companies are paying
barchart_company = base.mark_bar().encode(
    x = 'Company',
    y = 'Total_Payment',
    color=alt.Color('Nature_of_Payment', scale=alt.Scale(scheme='spectral')),
    tooltip= ['Number_of_Payments','Total_Payment','Nature_of_Payment']
).transform_filter(selection)

# Data on what drugs are being paid for.
barchart_drug = base.mark_bar().encode(
    x = 'Drug',
    y = 'Total_Payment',
    color=alt.Color('Nature_of_Payment', scale=alt.Scale(scheme='spectral')),
    tooltip= ['Number_of_Payments','Total_Payment','Therapeutic_Area','Nature_of_Payment']
).transform_filter(selection)

# Build the multi chart
scatterplot | barchart_company | barchart_drug





alt.HConcatChart(...)